In [1]:
import os
import numpy as np
import pandas as pd
import librosa
import soundfile as sf
from tqdm import tqdm
import audioread
import librosa.display
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve, recall_score, accuracy_score, precision_score
from scipy.signal import hilbert
from scipy.fftpack import fft, ifft
from scipy.stats import skew, kurtosis

# MFCC

In [2]:
# generate spectrograms
def plot_spectrogram(file_path):
    # Load the audio file
    y, sr = librosa.load(file_path)
    # resample the audio to 16kHz
    y = librosa.resample(y, orig_sr=sr, target_sr=16000)

    # Compute the Short-Time Fourier Transform (STFT)
    D = librosa.stft(y)

    # Convert to decibels (log scale) for better visualization
    D_dB = librosa.amplitude_to_db(np.abs(D), ref=np.max)

    # Plot the spectrogram
    plt.figure(figsize=(10, 5))
    librosa.display.specshow(D_dB, sr=sr, x_axis="time", y_axis="log", cmap="turbo")
    plt.colorbar(format="%+2.0f dB")
    plt.title("Spectrogram")
    plt.xlabel("Time (s)")
    plt.ylabel("Frequency (Hz)")
    plt.show()

def plot_mfcc(mfcc_dict, folder, file_name):
    """
    Plots the MFCC of a given file from the computed mfcc_dict.
    
    Args:
        mfcc_dict (dict): Dictionary containing MFCC features.
        folder (str): Folder name (e.g., 'dysarthria_female').
        file_name (str): Name of the audio file.
    """
    if folder not in mfcc_dict or file_name not in mfcc_dict[folder]:
        print("Error: Specified file not found in MFCC dictionary!")
        return
    
    mfcc = mfcc_dict[folder][file_name]  # Extract MFCC matrix

    # Plot MFCC
    plt.figure(figsize=(10, 5))
    librosa.display.specshow(mfcc, x_axis="time", y_axis="mel", cmap="magma")
    plt.colorbar(label="MFCC Coefficients")
    plt.title(f"MFCC - {file_name}")
    plt.xlabel("Time (s)")
    plt.ylabel("MFCC Coefficients")
    plt.show()

def plot_waveform(y, sr, title="Waveform"):
    plt.figure(figsize=(20, 4))
    librosa.display.waveshow(y, sr=sr)
    plt.title(title)
    # plt.xlim(0.3, 1.4)  # Set x-axis limit to duration of audio
    plt.xlabel("Time (s)")
    plt.ylabel("Amplitude")
    plt.show()

def show_waveplot(audio_path,label,gender):
    x , sr = librosa.load(audio_path)
    plt.figure(figsize=(15, 6))
    plt.grid(True)
    plt.xlabel("Time (s)")
    plt.ylabel("Amplitude")
    plt.xlim(0, len(x)/sr)  # Set x-axis limit to duration of audio
    librosa.display.waveshow(x, sr=sr)
    plt.title(f"Waveform of {label} speech: {gender}")

def show_spectrogram(audio_path,label,gender):
    x , sr = librosa.load(audio_path)
    X = librosa.stft(x)
    Xdb = librosa.amplitude_to_db(abs(X))
    plt.figure(figsize=(10,6))
    librosa.display.specshow(Xdb, sr=sr, x_axis='time', y_axis='hz',cmap='plasma')
    plt.colorbar()
    plt.title(f"Spectrogram of {label} speech: {gender}")
    
def show_zcr(audio_path,label,gender):
    x , sr = librosa.load(audio_path)
    zero_crossings = librosa.zero_crossings(x)
    print("Sum of zero crossing ", zero_crossings.sum())
    plt.figure(figsize=(20, 5))
    plt.title(f'Zero Crossing Rate of {label} speech: {gender}')
    zcrs = librosa.feature.zero_crossing_rate(x)
    plt.plot(zcrs[0])
    plt.show()
    
def show_mfccs(audio_path,label,gender):
    plt.figure(figsize=(20, 6))
    plt.title(f'MFCC of {label} speech: {gender}')
    x , sr = librosa.load(audio_path)
    mfccs = librosa.feature.mfcc(y=x, sr=sr)
    librosa.display.specshow(mfccs, sr=sr, x_axis='time',cmap='plasma')
    plt.show()
    
def show_melspectro(audio_path,label,gender):
    plt.figure(figsize=(20, 6))
    plt.title(f'Mel Spectro of {label} speech: {gender}')
    x , sr = librosa.load(audio_path)
    melspectro = librosa.feature.melspectrogram(y=x, sr=sr)
    librosa.display.specshow(melspectro, sr=sr, x_axis='time',cmap='plasma')
    plt.show()  

In [3]:
def feature_extraction(df):
    features = []
    labels = []

    for i, record in tqdm(df.iterrows(), total=df.shape[0]):
        try:
            x, sr = librosa.load(record['filename'])
            x = librosa.resample(x, orig_sr=sr, target_sr=16000)
            sr = 16000

            hop = np.floor(16000 * 0.008).astype(int)
            nfft = np.floor(16000 * 0.016).astype(int)
            mfccs = librosa.feature.mfcc(y=x, sr=sr, n_mfcc=13, n_fft=nfft, hop_length=hop)

            mfccs = mfccs.flatten()
            mfccs = np.nan_to_num(mfccs)  # Replace NaNs with 0

            features.append(mfccs)
            labels.append(record['is_dysarthria'])
        
        except Exception as e:
            print(f"Error with {record['filename']}: {e}")
    
    dataf = pd.DataFrame(features)
    dataf['class'] = labels

    # Convert string labels to numeric
    dataf.loc[dataf['class'] == 'non_dysarthria', 'class'] = 0.0
    dataf.loc[dataf['class'] == 'dysarthria', 'class'] = 1.0
    dataf['class'] = dataf['class'].astype(float)

    return dataf

In [6]:
data = pd.read_csv('modified_dataset/data.csv')  

In [7]:
dataf = feature_extraction(data)

  0%|          | 0/1999 [00:00<?, ?it/s]d:\Python\Miniconda\lib\site-packages\librosa\feature\spectral.py:2143: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  mel_basis = filters.mel(sr=sr, n_fft=n_fft, **kwargs)
100%|██████████| 1999/1999 [00:43<00:00, 45.87it/s]


In [8]:
dataf

,0,1,2,3,4,5,6,7,8,9,...,8831,8832,8833,8834,8835,8836,8837,8838,8839,class
0,-589.864136,-532.368469,-537.948486,-548.431274,-524.202942,-543.947388,-520.645874,-533.250244,-531.543884,-553.577759,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-573.136597,-539.961365,-558.506897,-547.889954,-574.227661,-552.250549,-573.702637,-558.173523,-570.032471,-555.576111,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-624.915710,-548.951904,-545.591736,-550.482666,-542.059204,-562.392883,-539.826782,-554.312195,-543.364258,-557.398865,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-569.522461,-546.824097,-538.010986,-550.043152,-527.377258,-545.896179,-528.217834,-556.866943,-527.705139,-549.090210,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-574.241455,-547.667847,-534.352600,-548.366821,-537.733276,-551.297668,-527.070251,-541.987000,-515.764099,-537.008301,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1994,-662.138794,-653.081299,-622.176331,-639.407959,-631.638977,-645.757690,-630.605103,-642.760742,-618.632385,-644.086121,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1995,-776.909729,-792.014465,-787.280640,-779.579529,-779.648682,-792.414124,-773.067017,-721.924438,-673.226501,-694.071411,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1996,-622.876282,-630.823730,-638.633057,-631.062500,-635.876831,-633.263916,-653.707581,-629.156006,-652.318359,-651.582153,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1997,-632.711426,-624.834900,-641.157654,-626.880798,-649.281128,-630.468262,-644.744202,-628.872437,-628.251526,-635.931824,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [27]:
dataf
# extract the 'class' column and save it to an array
# labels = dataf['class'].values

,0,1,2,3,4,5,6,7,8,9,...,41136,41137,41138,41139,41140,41141,41142,41143,41144,class
0,-589.831421,-532.361267,-537.957092,-548.404541,-524.203247,-543.943542,-520.645569,-533.228699,-531.529236,-553.609253,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-573.188049,-539.939697,-558.529541,-547.860840,-574.205994,-552.217529,-573.691345,-558.192932,-570.044250,-555.601013,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-624.855286,-548.967834,-545.632507,-550.492432,-542.005737,-562.411743,-539.829041,-554.259644,-543.355896,-557.421082,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-569.531067,-546.791565,-538.021851,-550.046875,-527.399902,-545.918030,-528.219055,-556.870544,-527.697205,-549.103516,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-574.218750,-547.657104,-534.359436,-548.373230,-537.745056,-551.300415,-527.102722,-542.008240,-515.769836,-537.032043,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1994,-662.105652,-653.149475,-622.161865,-639.379089,-631.657288,-645.737122,-630.571777,-642.720764,-618.652710,-644.047729,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1995,-777.129578,-791.894043,-787.382812,-779.727112,-779.752075,-792.240784,-773.137207,-721.311646,-673.268372,-694.012024,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1996,-622.753357,-630.777588,-638.582520,-631.112976,-635.816711,-633.183899,-653.693726,-629.097534,-652.379639,-651.664673,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1997,-632.678345,-624.834656,-641.152649,-626.907715,-649.286621,-630.512939,-644.792786,-628.906494,-628.243835,-635.951721,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [9]:
# drop 'class' column from the DataFrame
temp = dataf.drop(columns=['class'])
labels = dataf['class']
labels 

0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
1994    1.0
1995    1.0
1996    1.0
1997    1.0
1998    1.0
Name: class, Length: 1999, dtype: float64

In [10]:
# Number of columns per group
columns_per_group = 20

# Calculate the number of full groups (there may be some leftover columns)
num_groups = temp.shape[1] // columns_per_group
remaining_columns = temp.shape[1] % columns_per_group

# If there are remaining columns, we can add an extra group for them
if remaining_columns > 0:
    num_groups += 1

# Create an empty list to hold the averaged groups
grouped_data = []

# Iterate through each group of 40 columns (or fewer for the last group)
for i in range(num_groups):
    # Define the start and end column indices for this group
    start_col = i * columns_per_group
    end_col = min(start_col + columns_per_group, temp.shape[1])  # Ensure we don't go out of bounds

    # Slice the data for the current group and compute the mean along axis 1 (across columns)
    group = temp.iloc[:, start_col:end_col]  # Use .iloc for pandas slicing
    
    # Compute the mean across the columns (axis=1) for each group
    group_mean = group.mean(axis=1)
    
    # Append the averaged values to the list
    grouped_data.append(group_mean)

# Stack the grouped means to create the final feature vector
averaged_data = pd.DataFrame(np.column_stack(grouped_data))

# Verify the shape of the result (should be 1999 x 100)
print(averaged_data.shape)  # Output: (1999, 100)


(1999, 442)


In [11]:

#append the labels to the averaged data
averaged_data['class'] = labels.values
averaged_data

dataf_2 = averaged_data.copy()
dataf_2


,0,1,2,3,4,5,6,7,8,9,...,433,434,435,436,437,438,439,440,441,class
0,-541.014282,-535.843933,-536.986206,-539.283203,-533.296204,-532.315613,-533.949402,-503.632080,-524.486938,-508.439606,...,-5.229981,-0.999940,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
1,-559.230896,-557.901978,-559.004211,-555.190613,-557.237427,-554.392029,-493.430573,-401.234039,-399.868652,-554.415161,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
2,-560.130798,-556.864075,-559.008423,-555.559875,-561.693604,-557.824097,-520.016541,-361.092926,-449.393402,-514.996948,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
3,-544.344604,-540.748413,-543.488586,-545.004395,-541.683838,-540.879517,-392.618378,-422.350006,-367.769104,-365.009766,...,-1.742490,-5.340612,-3.398780,-9.058259,4.516374,-10.965194,-5.787478,-3.518082,-2.304311,0.0
4,-539.451904,-539.278198,-543.690613,-539.368469,-501.705566,-534.144470,-450.736176,-369.333954,-483.467224,-518.519897,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1994,-640.245850,-639.367920,-642.373108,-644.871948,-645.916382,-642.339233,-642.125061,-641.476685,-593.554321,-390.382385,...,0.116829,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0
1995,-746.640686,-665.395142,-740.651001,-556.258179,-465.729797,-492.057434,-501.072174,-513.504761,-547.957642,-593.843628,...,2.858545,2.833708,2.288659,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0
1996,-644.563904,-642.574707,-647.127136,-653.082092,-654.969360,-652.043884,-654.489990,-655.323853,-593.863159,-449.117279,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0
1997,-633.231812,-633.938232,-633.845886,-636.356445,-636.883118,-642.136902,-636.457703,-594.892822,-449.809418,-419.988281,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0


In [12]:
X = dataf_2.iloc[:,:-1].values
y = dataf_2.iloc[:,-1]

In [13]:
X.shape, y.shape

((1999, 442), (1999,))

## Results using MFCC

### a. SVM

In [14]:
# Normalize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Train SVM
svm_model = SVC(kernel='rbf', probability=True)
svm_model.fit(X, y)

# Predict and evaluate
y_pred = svm_model.predict(X)

print("SVM Accuracy:", accuracy_score(y, y_pred))
print("SVM Report:\n", classification_report(y, y_pred))
print("SVM Confusion Matrix:\n", confusion_matrix(y, y_pred))

SVM Accuracy: 0.9474737368684342
SVM Report:
               precision    recall  f1-score   support

         0.0       0.92      0.98      0.95      1000
         1.0       0.98      0.91      0.95       999

    accuracy                           0.95      1999
   macro avg       0.95      0.95      0.95      1999
weighted avg       0.95      0.95      0.95      1999

SVM Confusion Matrix:
 [[984  16]
 [ 89 910]]


### b. Random Forest

In [15]:
# Assuming X and y are your feature matrix and target variable respectively
# Split the data into training and testing sets (if not already done)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, stratify=y)
# Normalize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


# Train Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)  # You can adjust the number of estimators and other parameters
rf_model.fit(X_train, y_train)

# Predict and evaluate
y_pred = rf_model.predict(X_test)

# Evaluate the model
print("Random Forest Accuracy:", accuracy_score(y_test, y_pred))
print("Random Forest Report:\n", classification_report(y_test, y_pred))
print("Random Forest Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


Random Forest Accuracy: 0.93
Random Forest Report:
               precision    recall  f1-score   support

         0.0       0.96      0.90      0.93       100
         1.0       0.91      0.96      0.93       100

    accuracy                           0.93       200
   macro avg       0.93      0.93      0.93       200
weighted avg       0.93      0.93      0.93       200

Random Forest Confusion Matrix:
 [[90 10]
 [ 4 96]]


# ZTWCC

In [28]:
def pre_emphasis(signal, coeff=0.97):
    """Apply pre-emphasis to the signal"""
    return np.append(signal[0], signal[1:] - coeff * signal[:-1])

def decaying_window_w1(N):
    """Heavily decaying window"""
    w1 = np.zeros(N)
    for n in range(1, N):
        w1[n] = 1 / (4 * (np.sin(np.pi * n / (2 * N)))**2)
    return w1

def ripple_reducing_window_w2(M):
    """Square of half cosine window"""
    n = np.arange(M)
    return 4 * (np.cos(np.pi * n / (2 * M)))**2

def compute_ngd_spectrum(x, N):
    """Compute Numerator of Group Delay (NGD) spectrum"""
    x = np.pad(x, (0, N - len(x)), 'constant')
    y = np.arange(len(x)) * x

    X = fft(x, N)
    Y = fft(y, N)

    XR = np.real(X)
    XI = np.imag(X)
    YR = np.real(Y)
    YI = np.imag(Y)

    g = XR * YR + XI * YI
    return g

In [ ]:
def compute_ztwcc_stat_features(audio_path, L=5, N=512, fs=16000):
    try:
        # Load and pre-emphasize audio
        y, sr = librosa.load(audio_path, sr=None)
        if sr != fs:
            y = librosa.resample(y, orig_sr=sr, target_sr=fs)
        y = np.append(y[0], y[1:] - 0.97 * y[:-1])  # pre-emphasis

        M = int(L * fs / 1000)
        hop = M // 2
        num_frames = (len(y) - M) // hop + 1

        ztwccs = []

        for i in range(num_frames):
            start = i * hop
            frame = y[start:start + M]
            if len(frame) < M:
                break

            w1 = np.zeros(N)
            for n in range(1, N):
                w1[n] = 1 / (4 * (np.sin(np.pi * n / (2 * N)))**2)
            w2 = 4 * (np.cos(np.pi * np.arange(M) / (2 * M)))**2

            x = np.zeros(N)
            x[:M] = w2 * frame
            x = x * w1

            y_n = np.arange(N) * x
            X = fft(x, N)
            Y = fft(y_n, N)

            g = np.real(X) * np.real(Y) + np.imag(X) * np.imag(Y)
            g_diff2 = np.gradient(np.gradient(g))
            envelope = np.abs(hilbert(g_diff2))

            log_env = np.log(envelope + 1e-10)
            cepstrum = np.real(ifft(log_env))[:13]
            ztwccs.append(cepstrum)

        ztwccs = np.array(ztwccs)

        # Compute delta and delta-delta (using librosa functions)
        delta = librosa.feature.delta(ztwccs)
        delta2 = librosa.feature.delta(ztwccs, order=2)

        ztwcc_26D = np.concatenate((ztwccs, delta), axis = 1)
        ztwcc_39D = np.concatenate((ztwccs, delta, delta2), axis=1)

        # Statistical functionals
        feature_vector_39D = np.concatenate([
            np.mean(ztwcc_39D, axis=0),
            np.std(ztwcc_39D, axis=0),
            skew(ztwcc_39D, axis=0),
            kurtosis(ztwcc_39D, axis=0)
        ])

        feature_vector_26D = np.concatenate([
            np.mean(ztwcc_26D, axis=0),
            np.std(ztwcc_26D, axis=0),
            skew(ztwcc_26D, axis=0),
            kurtosis(ztwcc_26D, axis=0)
        ])

        feature_vector = np.concatenate([
            np.mean(ztwccs, axis=0),
            np.std(ztwccs, axis=0),
            skew(ztwccs, axis=0),
            kurtosis(ztwccs, axis=0)
        ])


        return feature_vector_26D

    except Exception as e:
        print(f"Error in {audio_path}: {e}")
        return None

In [94]:
x = compute_ztwcc_stat_features('torgo_data/non_dysarthria_male/MC01_Session1_0005.wav', L=5, N=512, fs=16000)
print("x shape:", x.shape)
# print("y shape:", y.shape)
# print("z shape:", z.shape)

x shape: (52,)


In [30]:
def ztwcc_feature_extraction(df):
    features = []
    labels = []

    for i, record in tqdm(df.iterrows(), total=df.shape[0]):
        feat = compute_ztwcc_stat_features(record['filename'])
        if feat is not None:
            features.append(feat)
            labels.append(record['is_dysarthria'])

    dataf = pd.DataFrame(features)
    dataf['class'] = labels

    # Convert string labels to numeric
    dataf.loc[dataf['class'] == 'non_dysarthria', 'class'] = 0.0
    dataf.loc[dataf['class'] == 'dysarthria', 'class'] = 1.0
    dataf['class'] = dataf['class'].astype(float)

    return dataf

In [32]:
ztwcc_df_26D = ztwcc_feature_extraction(data)

100%|██████████| 1999/1999 [2:58:10<00:00,  5.35s/it]  


In [20]:
# # save the features to a csv file
# ztwcc_df.to_csv('ztwcc_features.csv', index=False)
# save the features to a csv file
ztwcc_df_13D.to_csv('ztwcc_features_13D_v2.csv', index=False)

In [33]:
ztwcc_df_26D.to_csv('ztwcc_features_26D_v2.csv', index=False)

X = ztwcc_df_26D.iloc[:,:-1].values
y = ztwcc_df_26D.iloc[:,-1]
X.shape, y.shape

((1999, 104), (1999,))

In [21]:
X = ztwcc_df_13D.iloc[:,:-1].values
y = ztwcc_df_13D.iloc[:,-1]

In [22]:
X.shape, y.shape

((1999, 52), (1999,))

#### 13D features

### a. SVM

In [23]:
# Assume X (features) and y (labels) are already loaded
# For example:
# from sklearn.datasets import load_iris
# data = load_iris()
# X = data.data
# y = data.target

# --- Start of Changes ---

# 1. Split data into training and testing sets
# test_size=0.2 means 20% of the data will be used for testing, 80% for training
# random_state ensures reproducibility of the split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y if np.unique(y).size > 1 else None)

# 2. Normalize features
scaler = StandardScaler()

# Fit the scaler ONLY on the training data
scaler.fit(X_train)

# Transform both the training and testing data using the fitted scaler
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 3. Train SVM on the SCALED TRAINING data
svm_model = SVC(kernel='rbf', probability=True, random_state=42) # Added random_state for SVM reproducibility
svm_model.fit(X_train_scaled, y_train)

# 4. Predict on the SCALED TEST data
y_pred = svm_model.predict(X_test_scaled)

# 5. Evaluate using the TEST data labels and predictions
print("--- Evaluation on Test Set  for 13D---")
print("SVM Accuracy:", accuracy_score(y_test, y_pred))
print("SVM Classification Report:\n", classification_report(y_test, y_pred))
print("SVM Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

# --- End of Changes ---

# Optional: You could also evaluate on the training set to check for overfitting
# y_train_pred = svm_model.predict(X_train_scaled)
# print("\n--- Evaluation on Training Set ---")
# print("SVM Training Accuracy:", accuracy_score(y_train, y_train_pred))

--- Evaluation on Test Set  for 13D---
SVM Accuracy: 0.82
SVM Classification Report:
               precision    recall  f1-score   support

         0.0       0.79      0.86      0.83       200
         1.0       0.85      0.78      0.81       200

    accuracy                           0.82       400
   macro avg       0.82      0.82      0.82       400
weighted avg       0.82      0.82      0.82       400

SVM Confusion Matrix:
 [[173  27]
 [ 45 155]]


### b. Random Forest

In [24]:
# Assuming X and y are your feature matrix and target variable respectively
# Split the data into training and testing sets (if not already done)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)
# Normalize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


# Train Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)  # You can adjust the number of estimators and other parameters
rf_model.fit(X_train, y_train)

# Predict and evaluate
y_pred = rf_model.predict(X_test)

# Evaluate the model
print("Random Forest Accuracy:", accuracy_score(y_test, y_pred))
print("Random Forest Report:\n", classification_report(y_test, y_pred))
print("Random Forest Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Random Forest Accuracy: 0.87
Random Forest Report:
               precision    recall  f1-score   support

         0.0       0.86      0.89      0.87       200
         1.0       0.89      0.85      0.87       200

    accuracy                           0.87       400
   macro avg       0.87      0.87      0.87       400
weighted avg       0.87      0.87      0.87       400

Random Forest Confusion Matrix:
 [[178  22]
 [ 30 170]]


#### 26D features

In [105]:
ztwcc_df_26D = pd.read_csv('ztwcc_features_26D.csv')
ztwcc_df_26D

,0,1,2,3,4,5,6,7,8,9,...,95,96,97,98,99,100,101,102,103,class
0,9.007761,0.210391,-0.284657,0.207617,-0.169936,0.155279,-0.141293,0.133329,-0.105965,0.055164,...,-1.055259,-0.120639,0.194613,0.718923,0.497148,0.497148,0.497148,0.497148,0.497148,0.0
1,7.210843,0.163901,-0.303886,0.175621,-0.206025,0.180544,-0.151549,0.102758,-0.125537,0.068917,...,0.346101,1.317038,1.835086,0.711150,0.669803,0.669803,0.669803,0.669803,0.669803,0.0
2,7.059375,0.115556,-0.350706,0.181913,-0.187130,0.167360,-0.147741,0.102714,-0.122170,0.060719,...,1.811681,0.340054,0.606853,1.013814,3.379625,3.379625,3.379625,3.379625,3.379625,0.0
3,9.291084,0.262941,-0.260902,0.163837,-0.180458,0.141870,-0.204044,0.105289,-0.140475,0.048670,...,-1.207347,-0.650605,0.068324,0.606972,1.117118,1.117118,1.117118,1.117118,1.117118,0.0
4,7.705002,0.113418,-0.311381,0.197590,-0.196407,0.158966,-0.141373,0.119998,-0.105476,0.074076,...,1.165513,0.945998,0.968477,2.154683,3.610434,3.610434,3.610434,3.610434,3.610434,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1994,6.671825,0.420362,-0.190024,0.197186,-0.132535,0.144535,-0.122294,0.121546,-0.091930,0.070988,...,-1.180815,0.401496,1.016533,1.011775,0.705707,0.705707,0.705707,0.705707,0.705707,1.0
1995,4.154088,0.251432,0.162146,0.188040,-0.044941,-0.021588,0.000835,-0.026123,-0.100122,-0.009276,...,-1.070262,-1.211949,-0.547511,-0.762649,-0.228822,-0.228822,-0.228822,-0.228822,-0.228822,1.0
1996,5.220163,0.396065,-0.155141,0.162118,-0.136280,0.152045,-0.130485,0.112906,-0.100571,0.066224,...,0.684507,1.651040,1.792216,0.759141,2.483550,2.483550,2.483550,2.483550,2.483550,1.0
1997,6.655282,0.451367,-0.227539,0.123284,-0.131509,0.190148,-0.099268,0.135190,-0.083929,0.058102,...,-1.338496,0.085416,0.668628,0.571975,0.771213,0.771213,0.771213,0.771213,0.771213,1.0


In [34]:
X = ztwcc_df_26D.iloc[:,:-1].values
y = ztwcc_df_26D.iloc[:,-1]

X.shape, y.shape

((1999, 104), (1999,))

### SVM

In [35]:
# Assume X (features) and y (labels) are already loaded
# For example:
# from sklearn.datasets import load_iris
# data = load_iris()
# X = data.data
# y = data.target

# --- Start of Changes ---

# 1. Split data into training and testing sets
# test_size=0.2 means 20% of the data will be used for testing, 80% for training
# random_state ensures reproducibility of the split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y if np.unique(y).size > 1 else None)

# 2. Normalize features
scaler = StandardScaler()

# Fit the scaler ONLY on the training data
scaler.fit(X_train)

# Transform both the training and testing data using the fitted scaler
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 3. Train SVM on the SCALED TRAINING data
svm_model = SVC(kernel='rbf', probability=True, random_state=42) # Added random_state for SVM reproducibility
svm_model.fit(X_train_scaled, y_train)

# 4. Predict on the SCALED TEST data
y_pred = svm_model.predict(X_test_scaled)

# 5. Evaluate using the TEST data labels and predictions
print("--- Evaluation on Test Set  for 26D---")
print("SVM Accuracy:", accuracy_score(y_test, y_pred))
print("SVM Classification Report:\n", classification_report(y_test, y_pred))
print("SVM Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

# --- End of Changes ---

# Optional: You could also evaluate on the training set to check for overfitting
# y_train_pred = svm_model.predict(X_train_scaled)
# print("\n--- Evaluation on Training Set ---")
# print("SVM Training Accuracy:", accuracy_score(y_train, y_train_pred))

--- Evaluation on Test Set  for 26D---
SVM Accuracy: 0.8425
SVM Classification Report:
               precision    recall  f1-score   support

         0.0       0.83      0.86      0.85       200
         1.0       0.85      0.82      0.84       200

    accuracy                           0.84       400
   macro avg       0.84      0.84      0.84       400
weighted avg       0.84      0.84      0.84       400

SVM Confusion Matrix:
 [[172  28]
 [ 35 165]]


### Random Forest

In [36]:
# Assuming X and y are your feature matrix and target variable respectively
# Split the data into training and testing sets (if not already done)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)
# Normalize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


# Train Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)  # You can adjust the number of estimators and other parameters
rf_model.fit(X_train, y_train)

# Predict and evaluate
y_pred = rf_model.predict(X_test)

# Evaluate the model
print("Random Forest Accuracy:", accuracy_score(y_test, y_pred))
print("Random Forest Report:\n", classification_report(y_test, y_pred))
print("Random Forest Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Random Forest Accuracy: 0.8925
Random Forest Report:
               precision    recall  f1-score   support

         0.0       0.89      0.90      0.89       200
         1.0       0.90      0.89      0.89       200

    accuracy                           0.89       400
   macro avg       0.89      0.89      0.89       400
weighted avg       0.89      0.89      0.89       400

Random Forest Confusion Matrix:
 [[180  20]
 [ 23 177]]


#### 39D features

### a. SVM

In [116]:
ztwcc_df_39D = pd.read_csv('ztwcc_features.csv')
ztwcc_df_39D

,0,1,2,3,4,5,6,7,8,9,...,147,148,149,150,151,152,153,154,155,class
0,9.007761,0.210391,-0.284657,0.207617,-0.169936,0.155279,-0.141293,0.133329,-0.105965,0.055164,...,-0.887010,0.638045,1.140946,1.322222,1.527798,1.527798,1.527798,1.527798,1.527798,0.0
1,7.210843,0.163901,-0.303886,0.175621,-0.206025,0.180544,-0.151549,0.102758,-0.125537,0.068917,...,0.397660,3.076936,0.615267,0.112652,1.150858,1.150858,1.150858,1.150858,1.150858,0.0
2,7.059375,0.115556,-0.350706,0.181913,-0.187130,0.167360,-0.147741,0.102714,-0.122170,0.060719,...,2.104914,1.759690,0.965853,0.599206,0.252974,0.252974,0.252974,0.252974,0.252974,0.0
3,9.291084,0.262941,-0.260902,0.163837,-0.180458,0.141870,-0.204044,0.105289,-0.140475,0.048670,...,-1.199495,-0.362526,-0.159222,0.618206,0.164432,0.164432,0.164432,0.164432,0.164432,0.0
4,7.705002,0.113418,-0.311381,0.197590,-0.196407,0.158966,-0.141373,0.119998,-0.105476,0.074076,...,1.682025,1.547169,1.202286,0.749581,0.811814,0.811814,0.811814,0.811814,0.811814,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1994,6.671825,0.420362,-0.190024,0.197186,-0.132535,0.144535,-0.122294,0.121546,-0.091930,0.070988,...,-1.160842,0.582556,1.367601,1.751465,1.373939,1.373939,1.373939,1.373939,1.373939,1.0
1995,4.154088,0.251432,0.162146,0.188040,-0.044941,-0.021588,0.000835,-0.026123,-0.100122,-0.009276,...,-0.933690,-0.824036,-0.046051,-0.474372,0.114853,0.114853,0.114853,0.114853,0.114853,1.0
1996,5.220163,0.396065,-0.155141,0.162118,-0.136280,0.152045,-0.130485,0.112906,-0.100571,0.066224,...,1.323658,1.113896,2.595700,1.487860,0.319635,0.319635,0.319635,0.319635,0.319635,1.0
1997,6.655282,0.451367,-0.227539,0.123284,-0.131509,0.190148,-0.099268,0.135190,-0.083929,0.058102,...,-1.090845,0.816643,0.513567,0.460863,1.104019,1.104019,1.104019,1.104019,1.104019,1.0


In [118]:
X = ztwcc_df_39D.iloc[:,:-1].values
y = ztwcc_df_39D.iloc[:,-1]

X.shape, y.shape

((1999, 156), (1999,))

In [120]:
# Assume X (features) and y (labels) are already loaded
# For example:
# from sklearn.datasets import load_iris
# data = load_iris()
# X = data.data
# y = data.target

# --- Start of Changes ---

# 1. Split data into training and testing sets
# test_size=0.2 means 20% of the data will be used for testing, 80% for training
# random_state ensures reproducibility of the split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y if np.unique(y).size > 1 else None)

# 2. Normalize features
scaler = StandardScaler()

# Fit the scaler ONLY on the training data
scaler.fit(X_train)

# Transform both the training and testing data using the fitted scaler
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 3. Train SVM on the SCALED TRAINING data
svm_model = SVC(kernel='rbf', probability=True, random_state=42) # Added random_state for SVM reproducibility
svm_model.fit(X_train_scaled, y_train)

# 4. Predict on the SCALED TEST data
y_pred = svm_model.predict(X_test_scaled)

# 5. Evaluate using the TEST data labels and predictions
print("--- Evaluation on Test Set  for 39D---")
print("SVM Accuracy:", accuracy_score(y_test, y_pred))
print("SVM Classification Report:\n", classification_report(y_test, y_pred))
print("SVM Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

# --- End of Changes ---

# Optional: You could also evaluate on the training set to check for overfitting
# y_train_pred = svm_model.predict(X_train_scaled)
# print("\n--- Evaluation on Training Set ---")
# print("SVM Training Accuracy:", accuracy_score(y_train, y_train_pred))

--- Evaluation on Test Set  for 39D---
SVM Accuracy: 0.925
SVM Classification Report:
               precision    recall  f1-score   support

         0.0       0.91      0.94      0.93       200
         1.0       0.94      0.91      0.92       200

    accuracy                           0.93       400
   macro avg       0.93      0.93      0.92       400
weighted avg       0.93      0.93      0.92       400

SVM Confusion Matrix:
 [[189  11]
 [ 19 181]]


In [121]:
# Assuming X and y are your feature matrix and target variable respectively
# Split the data into training and testing sets (if not already done)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)
# Normalize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


# Train Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)  # You can adjust the number of estimators and other parameters
rf_model.fit(X_train, y_train)

# Predict and evaluate
y_pred = rf_model.predict(X_test)

# Evaluate the model
print("Random Forest Accuracy:", accuracy_score(y_test, y_pred))
print("Random Forest Report:\n", classification_report(y_test, y_pred))
print("Random Forest Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Random Forest Accuracy: 0.9275
Random Forest Report:
               precision    recall  f1-score   support

         0.0       0.91      0.95      0.93       200
         1.0       0.95      0.90      0.93       200

    accuracy                           0.93       400
   macro avg       0.93      0.93      0.93       400
weighted avg       0.93      0.93      0.93       400

Random Forest Confusion Matrix:
 [[191   9]
 [ 20 180]]


# SFCCs

In [ ]:
def pre_emphasis(signal):
    return np.append(signal[0], signal[1:] - signal[:-1])

def frequency_shift(x, fk, fs):
    N = len(x)
    n = np.arange(N)
    w_k_bar = np.pi - 2 * np.pi * fk / fs
    return x * np.exp(1j * w_k_bar * n)

def single_pole_filter(x_kn, r=0.9875):
    y = np.zeros_like(x_kn, dtype=np.complex128)
    for n in range(1, len(x_kn)):
        y[n] = -r * y[n - 1] + x_kn[n]
    return y

def amplitude_envelope(y_kn):
    return np.real(y_kn)**2 + np.imag(y_kn)**2

def compute_sffcc(sff_envelopes):
    log_spectrum = np.log(sff_envelopes + 1e-8)
    cepstrum = np.real(ifft(log_spectrum, axis=0))
    return cepstrum

def extract_sffcc(signal, fs, fk_range, r=0.9875):
    emphasized = pre_emphasis(signal)
    N = len(emphasized)
    sff_envs = []

    for fk in fk_range:
        x_kn = frequency_shift(emphasized, fk, fs)
        y_kn = single_pole_filter(x_kn, r)
        v_kn = amplitude_envelope(y_kn)
        sff_envs.append(v_kn)
    
    sff_envs = np.array(sff_envs)
    sffcc = compute_sffcc(sff_envs)
    
    return sffcc.T 

In [ ]:
def sffcc_feature_extraction(df, fk_range, fs=16000, r=0.9875):
    features = []
    labels = []

    for i, record in tqdm(df.iterrows(), total=df.shape[0]):
        try:
            # Load audio
            signal, sr = librosa.load(record['filename'], sr=None)
            if sr != fs:
                signal = librosa.resample(signal, orig_sr=sr, target_sr=fs)

            # Extract SFFCCs
            sffcc = extract_sffcc(signal, fs, fk_range, r=r)  # shape: [time_frames, num_ceps]
   
            # Calculate mean, std, skew, kurtosis per row (frame)
            stats = []
            for row in tqdm(sffcc):
                row_mean = np.mean(row)
                row_std = np.std(row)
                row_skew = skew(row)
                row_kurt = kurtosis(row)
                stats.append([row_mean, row_std, row_skew, row_kurt])

            # Convert to 2D array [num_frames, 4]
            stat_array = np.array(stats)
            # Make the stat_array 1D
            stat_array = stat_array.flatten()  # optional: flatten if needed
            features.append(stat_array)  # optional: could also flatten each row if needed
            labels.append(record['is_dysarthria'])

        except Exception as e:
            print(f"Error processing {record['filename']}: {e}")

    return features, labels  # returns a list of 2D arrays and corresponding labels


In [136]:
# Define frequency range
num_bins = 400
fk_range = np.linspace(0, 8000, num_bins, endpoint=False)

In [137]:
sffcc_df = sffcc_feature_extraction(data, fk_range)

  0%|          | 2/1999 [02:49<47:04:26, 84.86s/it]


KeyboardInterrupt: 

In [132]:
from scipy.io import wavfile

In [134]:
fs, audio = wavfile.read('sample_non_dysarthria.wav')
if audio.ndim > 1:
    audio = audio[:, 0]  # Use first channel if stereo

audio = audio / np.max(np.abs(audio))  # Normalize

# Define frequency range
num_bins = 400
fk_range = np.linspace(0, 8000, num_bins, endpoint=False)

# Extract SFFCCs
sffccs = extract_sffcc(audio, 16000, fk_range, r=0.995)
sffccs.shape

(51280, 400)

In [ ]:
np.mean()